# Metadata

**Customer Domain:** - Chemical Manufacturing  
  
**Customer Subdomain:** - Chemical Manufacturing  
  
**Domain:** - Complex Instruction Following  
  
**Taxonomy Type:** - Tabular Data  
  
**L2 Taxonomy Type:** -  Data Transformation (Normalization/Standardization)  
  
**Conversation Length:** - 3 Turn Tasks  
  
**Scenario:** - [Company], a specialty chemical manufacturer, needs to optimize the mixing process for a new polymer blend, [PolymerBlendX]. They've been collecting batch data from various sensors across their reactor. However, this data is currently stored in a fragmented and inconsistent manner, hindering effective analysis. Specifically, the input data comes from two sources: Source A, a legacy system, reports temperature readings in Celsius with varying levels of precision (e.g., "25.1", "100", "78.675"), and pressure in PSI rounded to the nearest integer. Source B, a newer IoT sensor network, provides temperature in Fahrenheit with two decimal places of precision and pressure in bar, also with two decimal places. These readings are recorded as strings within comma separated value files, with the first column indicating the timestamp in 'YYYY-MM-DD HH:MM:SS' format and inconsistent column order across files.  
  
To effectively analyze the batch data and optimize [PolymerBlendX]'s mixing process, [Company] requires a data transformation pipeline. This pipeline must accept the raw CSV strings from both Source A and Source B. The data needs to be normalized, converting all temperature readings to a common unit (Kelvin) and pressure readings to a common unit (Pascal). Furthermore, the varying precision needs to be standardized to two decimal places for both temperature and pressure, and the timestamp format consolidated. Finally, the transformed data, including timestamp, standardized temperature, and standardized pressure, should be presented in a single, unified tabular format suitable for further analysis by data scientists.  
  


**[system]**

---

You are an AI assistant helping a chemical manufacturer prepare and normalize fragmented batch sensor data from multiple sources. The goal is to support optimizing the mixing process for their new polymer blend. The user will provide one or more datasets directly into the conversation. You are responsible for validating the input, clarifying unit formats, detecting anomalies, standardizing the data, and supporting optional data segmentation or cleanup.

When the user initiates a request, begin by asking them to share the CSV-formatted data. Each dataset must include at least three columns: a timestamp, a temperature value, and a pressure value. If any column is missing, ask the user to provide the required columns. The timestamp column may appear in different formats, so after receiving the data, ask the user to describe the format used. For example, a user might say the timestamp format is "YYYY-MM-DD HH:MM:SS" or "DD/MM/YYYY HH:MM".
If the user shares example rows to illustrate the timestamp format, inspect the examples. Confirm that the date in the example rows matches the date in the actual dataset. If the dates match, determine the timestamp format based on the examples and parse accordingly. If the dates do not align or are unclear, ask the user for clarification before proceeding.

The temperature and pressure columns may or may not specify the units in their headers. If the units are mentioned, such as "Temperature_C" or "Pressure_bar", extract them directly. If not, ask the user to clarify which units were used for each relevant column. Only temperature values in Celsius or Fahrenheit are accepted. If the user provides any other temperature unit, inform them that it is not supported, and you cannot proceed until a supported unit is confirmed. Only pressure values in PSI or bar are accepted. If any other pressure unit is used, inform the user that you cannot continue until a valid unit is provided.

After getting the datasets and necessary inputs, scan for rows that contain missing or invalid values. If any non-numeric data appears in the temperature or pressure columns, flag those rows. If any timestamp values cannot be parsed based on the provided format, inform the user and show the affected entries. Also, detect and report duplicate timestamps so that the user can decide whether to keep or remove them.

Once the data passes validation, proceed with standardization. Convert all temperature values to Kelvin. If the original unit is Celsius, use the formula:
$$  
K = C + 273.15  
$$
If the original unit is Fahrenheit, use the formula:
$$  
K = \left(\frac{F - 32}{1.8}\right) + 273.15  
$$
For pressure, convert all values to Pascal. If the original unit is PSI, use  
$$
Pa = PSI \times 6894.76  
$$
If the unit is a bar, use  
$$
Pa = bar \times 100000  
$$

If the temperature is in Kelvin, or the pressure is in Pascal, no conversion is required, only perform rounding if necessary. However, if the user provides temperature or pressure values on a larger scale such as kilopascal (kPa), convert it to Pascal using the following formula:
$$
Pa = kPa \times 1000
$$

After conversion, round all temperature and pressure values to two decimal places. Reformat all timestamps to the unified format "YYYY-MM-DD HH:MM:SS".

After standardization, perform anomaly detection. Use the following thresholds for detecting outliers: temperature values in Kelvin must be between 250 and 700, and pressure values in Pascal must fall between 50000 and 2000000. Any row that contains a value outside these ranges should be flagged as a suspected anomaly. Present these rows to the user along with their row number, timestamp, and offending values, and ask the user whether to keep or remove them from the final dataset.

Present the calculation and standardization in the following format, starting with an intro line giving an overview of the processing:
```markdown
[For each dataset]
### Dataset [Dataset Number] (Temperature in [unit], Pressure in [unit])  
Formulas used:  
[Present the formula]

| timestamp   | temperature ([unit]) | pressure ([unit]) | temperature_K calculation   | pressure_Pa calculation   |
|-------------|----------------------|-------------------|-----------------------------|---------------------------|
| [timestamp] | [temperature]        | [pressure]        | [temperature_K calculation] | [pressure_Pa calculation] |

### Combined Dataset (Temperature in Kelvin, Pressure in Pascal, sorted by timestamp)  

| timestamp   | temperature_K   | pressure_Pa   |  
|-------------|-----------------|---------------|  
| [timestamp] | [temperature_K] | [pressure_Pa] |
  
### Anomaly Check  
  
Thresholds:  
- Temperature in Kelvin must be between 250 and 700  
- Pressure in Pascal must be between 50000 and 2000000  
  
[Check for the anomaly based on the threshold and inform the user if the dataset is ready for further processing.]
```

After presenting the combined dataset, ask follow-up questions if the user wants an optional enhancement. You can segment the dataset based on specific timestamp ranges, such as daily intervals, work shifts, or custom date ranges provided by the user. If the batch_id column is present, you can also segment the data by batch. Additionally, offer to handle missing values, if any, by applying imputation strategies, such as using the column mean, median, or forward-fill logic, depending on what the user prefers.

**[user]**

hey so uh... we ran some batches last week and the data's kinda all over the place. got two files here - one from the morning shift (total mess) and one from night shift (looks cleaner?). can you fix this up for the polymer reports?

File1 (morning shift data - looks weird af):
batch_id,timestamp,temp_°R,pressure_mmHg
B1,05/06/2023 07:00, 512, 760
B1,05/06/2023 07:15, 515, 765
B1,05/06/2023 07:30, 518, 770
B1,05/06/2023 07:45, 521, 775
B1,05/06/2023 08:00, 524, 780
B1,05/06/2023 08:15, 527, 785
B1,05/06/2023 08:30, 530, 790
B1,05/06/2023 08:45, 533, 795
B1,05/06/2023 09:00, 536, 800
B1,05/06/2023 09:15, 539, 805

File2 (night shift - should be better?):
batch_id,time,temp_°C,pressure_kPa
B2,2023-05-06 21:00:00, 82, 110
B2,2023-05-06 21:15:00, 83, 111
B2,2023-05-06 21:30:00, 84, 112
B2,2023-05-06 21:45:00, 85, 113
B2,2023-05-06 22:00:00, 86, 114
B2,2023-05-06 22:15:00, 87, 115
B2,2023-05-06 22:30:00, 88, 116
B2,2023-05-06 22:45:00, 89, 117
B2,2023-05-06 23:00:00, 90, 118
B2,2023-05-06 23:15:00, 91, 119

so like... the morning guys used some ancient equipment i guess? temp is in °R (rankine??) and pressure in mmHg. night shift used normal units i think. we need everything in kelvin and pascals for the final report. also the timestamps are formatted differently lol. can you clean this up and tell me if the data looks ok? thx!

**[assistant]**

The morning shift file has some issues that need to be addressed before we can proceed:

- The temperature unit is Rankine (°R), which is not supported. Only Celsius or Fahrenheit are accepted.

- The pressure unit is millimeters of mercury (mmHg), which is not supported. Only PSI or bar are accepted.

The night shift file uses Celsius for temperature and kilopascal for pressure, which are acceptable.

Also, the timestamp formats are different between the two files. Could you please confirm the format used in each file? For example, is it "DD/MM/YYYY HH:MM" for the morning shift and "YYYY-MM-DD HH:MM:SS" for the night shift?  
I want to make sure I parse them correctly before proceeding.

**[user]**

here's the fixed morning shift data. converted that wacky rankine stuff to celsius and mmHg to bar like you asked:

batch_id,timestamp,temp_C,pressure_bar
B1,05/06/2023 07:00, 28.33, 1.013
B1,05/06/2023 07:15, 28.89, 1.020
B1,05/06/2023 07:30, 29.44, 1.027
B1,05/06/2023 07:45, 30.00, 1.033
B1,05/06/2023 08:00, 30.56, 1.040
B1,05/06/2023 08:15, 31.11, 1.047
B1,05/06/2023 08:30, 31.67, 1.053
B1,05/06/2023 08:45, 32.22, 1.060
B1,05/06/2023 09:00, 32.78, 1.067
B1,05/06/2023 09:15, 33.33, 1.073

night shift data is same as before. can you process both now? also the date format is MM/DD/YYYY. thx!

**[assistant]**

Thank you for the updated morning shift data. The temperature and pressure units are now acceptable.

You mentioned that the date format is MM/DD/YYYY. However, the night shift data you shared earlier appears to use a different format. Could you please confirm the timestamp format for the night shift file?  

**[user]**

ugh, timestamps are annoying. night shift file is whatever excel spat out - think it's "Month Day, Year HH:MM" format. here's the first few rows again if that helps:

Batch ID,Timestamp,Temp,Press
B2,May 6, 2023 21:00,82,110
B2,May 6, 2023 21:15,83,111

just make everything consistent or whatever. also can you show me what the final processed data looks like for both files? need to send to my boss in an hour lol

**[assistant]**

I see you mentioned that the night shift timestamps were exported by Excel and might be in the "Month Day, Year HH:MM" format.
You also shared the first two rows:

```csv
Batch ID,Timestamp,Temp,Press
B2,May 6, 2023 21:00,82,110
B2,May 6, 2023 21:15,83,111
```

Based on these rows, I can confirm that the date is May 6, 2023.
Since the earlier night shift file you shared already contains timestamps like "2023-05-06 21:00:00", I can parse the night shift data using the "YYYY-MM-DD HH:MM:SS" format, treating May 6 accordingly.

For the morning shift data, the timestamps follow the "MM/DD/YYYY HH:MM" format.

After parsing each file according to its correct format, I will standardize all timestamps to "YYYY-MM-DD HH:MM:SS".

Now processing both datasets.

### Dataset 1 (Temperature in Celsius, Pressure in bar)  
Formulas used:  
- Temperature in Kelvin:  
$$  
K = C + 273.15  
$$  
- Pressure in Pascal:  
$$  
Pa = bar \times 100000  
$$  

| timestamp           | temperature (Celsius) | pressure (bar) | temperature_K calculation | pressure_Pa calculation |
|---------------------|------------------------|----------------|----------------------------|--------------------------|
| 05/06/2023 07:00     | 28.33                  | 1.013          | 301.48                    | 101300.00                |
| 05/06/2023 07:15     | 28.89                  | 1.020          | 302.04                    | 102000.00                |
| 05/06/2023 07:30     | 29.44                  | 1.027          | 302.59                    | 102700.00                |
| 05/06/2023 07:45     | 30.00                  | 1.033          | 303.15                    | 103300.00                |
| 05/06/2023 08:00     | 30.56                  | 1.040          | 303.71                    | 104000.00                |
| 05/06/2023 08:15     | 31.11                  | 1.047          | 304.26                    | 104700.00                |
| 05/06/2023 08:30     | 31.67                  | 1.053          | 304.82                    | 105300.00                |
| 05/06/2023 08:45     | 32.22                  | 1.060          | 305.37                    | 106000.00                |
| 05/06/2023 09:00     | 32.78                  | 1.067          | 305.93                    | 106700.00                |
| 05/06/2023 09:15     | 33.33                  | 1.073          | 306.48                    | 107300.00                |

### Dataset 2 (Temperature in Celsius, Pressure in kilopascal)  
Formulas used:  
- Temperature in Kelvin:  
$$  
K = C + 273.15  
$$  
- Pressure in Pascal:  
$$  
Pa = kPa \times 1000  
$$  

| timestamp           | temperature (Celsius) | pressure (kPa) | temperature_K calculation | pressure_Pa calculation |
|---------------------|------------------------|----------------|----------------------------|--------------------------|
| 2023-05-06 21:00:00  | 82                     | 110            | 355.15                    | 110000.00                |
| 2023-05-06 21:15:00  | 83                     | 111            | 356.15                    | 111000.00                |
| 2023-05-06 21:30:00  | 84                     | 112            | 357.15                    | 112000.00                |
| 2023-05-06 21:45:00  | 85                     | 113            | 358.15                    | 113000.00                |
| 2023-05-06 22:00:00  | 86                     | 114            | 359.15                    | 114000.00                |
| 2023-05-06 22:15:00  | 87                     | 115            | 360.15                    | 115000.00                |
| 2023-05-06 22:30:00  | 88                     | 116            | 361.15                    | 116000.00                |
| 2023-05-06 22:45:00  | 89                     | 117            | 362.15                    | 117000.00                |
| 2023-05-06 23:00:00  | 90                     | 118            | 363.15                    | 118000.00                |
| 2023-05-06 23:15:00  | 91                     | 119            | 364.15                    | 119000.00                |

### Combined Dataset (Temperature in Kelvin, Pressure in Pascal, sorted by timestamp)

| timestamp           | temperature_K   | pressure_Pa   |
|---------------------|------------------|---------------|
| 2023-05-06 07:00:00  | 301.48            | 101300.00     |
| 2023-05-06 07:15:00  | 302.04            | 102000.00     |
| 2023-05-06 07:30:00  | 302.59            | 102700.00     |
| 2023-05-06 07:45:00  | 303.15            | 103300.00     |
| 2023-05-06 08:00:00  | 303.71            | 104000.00     |
| 2023-05-06 08:15:00  | 304.26            | 104700.00     |
| 2023-05-06 08:30:00  | 304.82            | 105300.00     |
| 2023-05-06 08:45:00  | 305.37            | 106000.00     |
| 2023-05-06 09:00:00  | 305.93            | 106700.00     |
| 2023-05-06 09:15:00  | 306.48            | 107300.00     |
| 2023-05-06 21:00:00  | 355.15            | 110000.00     |
| 2023-05-06 21:15:00  | 356.15            | 111000.00     |
| 2023-05-06 21:30:00  | 357.15            | 112000.00     |
| 2023-05-06 21:45:00  | 358.15            | 113000.00     |
| 2023-05-06 22:00:00  | 359.15            | 114000.00     |
| 2023-05-06 22:15:00  | 360.15            | 115000.00     |
| 2023-05-06 22:30:00  | 361.15            | 116000.00     |
| 2023-05-06 22:45:00  | 362.15            | 117000.00     |
| 2023-05-06 23:00:00  | 363.15            | 118000.00     |
| 2023-05-06 23:15:00  | 364.15            | 119000.00     |

### Anomaly Check

Thresholds:
- Temperature in Kelvin must be between 250 and 700
- Pressure in Pascal must be between 50000 and 2000000

No anomalies detected. The dataset is ready for further processing.


Would you like me to help you segment the data by shift or day before you send it?  